<a href="https://colab.research.google.com/github/YonggunJung/ICT_TensorFlow/blob/main/20230918.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Logistic Regression 구현
# Titanic(Kaggle)

# 필요한 module import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam

# Train data와 Test data를 분리하지 않는다.
# Test data를 Kaggle 에서 제공



In [9]:
# Raw Data Loading
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICT Innovation Square/data/titanic_train.csv')
# display(df)
# 	PassengerId : 승객 번호 =>삭제
# 	Survived : 생존여부 (0:사망, 1:생존) => 종속 변수
#   Pclass	: 1, 2, 3(1등급이 가장 좋음, 객실 등급)
#   Name : 이름 =>삭제
#   Sex : 성별
#   Age : 나이
#   SibSp : 형제와 배우자
#   Parch : 부모와 자식
#   Ticket : 티켓번호 =>삭제
#   Fare : 요금 =>삭제
#   Cabin : 객실번호 =>삭제
#   Embarked : 승선한 곳 - 상식적으로 삭제 대상이 맞으나 상관관계 분석을 해보면 영향이 있다

train = df

# 사용하지 않는 컬럼 제거
train.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis = 1, inplace = True)
# display(train)

# 성별처리 male = 0, female = 1  (매핑 mapping)
gender_mapping = {'male':0, 'female':1}
train['Sex'] = train['Sex'].map(gender_mapping)

# 가족처리
train['Family'] = train['SibSp'] + train['Parch']
train.drop(['SibSp', 'Parch'], axis = 1, inplace = True)

# Embarked 결측치 처리
train['Embarked'] = train['Embarked'].fillna('Q')

embarked_mapping = {'S':0, 'C':1, 'Q':2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)

# Age 결측치 처리 평균값으로 대체
train['Age'] = train['Age'].fillna(train['Age'].mean())

# Age Binning 처리(Numerical Value -> Categorical Value)
train.loc[train['Age'] < 8, 'Age'] = 0
train.loc[(train['Age'] >= 8) & (train['Age'] < 20), 'Age'] = 1
train.loc[(train['Age'] >= 20) & (train['Age'] < 65), 'Age'] = 2
train.loc[train['Age'] >= 65, 'Age'] = 3

display(train)

x_data = train.drop('Survived', axis = 1, inplace = False).values
t_data = train['Survived'].values.reshape(-1, 1)

,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,2.0,0,1
1,1,1,1,2.0,1,1
2,1,3,1,2.0,0,0
3,1,1,1,2.0,0,1
4,0,3,0,2.0,0,0
...,...,...,...,...,...,...
886,0,2,0,2.0,0,0
887,1,1,1,1.0,0,0
888,0,3,1,2.0,0,3
889,1,1,0,2.0,1,0


In [13]:
model = Sequential()

model.add(Flatten(input_shape = (5,)))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = Adam(learning_rate = 1e-2), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x_data, t_data, epochs = 300, verbose = 1, validation_split = 0.3)

Epoch 1/300
20/20 [==============================] - 1s 14ms/step - loss: 1.2881 - accuracy: 0.3965 - val_loss: 1.0602 - val_accuracy: 0.3993
Epoch 2/300
20/20 [==============================] - 0s 4ms/step - loss: 0.8572 - accuracy: 0.4430 - val_loss: 0.7477 - val_accuracy: 0.6045
Epoch 3/300
20/20 [==============================] - 0s 4ms/step - loss: 0.6843 - accuracy: 0.6437 - val_loss: 0.6234 - val_accuracy: 0.7127
Epoch 4/300
20/20 [==============================] - 0s 4ms/step - loss: 0.6152 - accuracy: 0.7079 - val_loss: 0.5734 - val_accuracy: 0.7425
Epoch 5/300
20/20 [==============================] - 0s 4ms/step - loss: 0.5762 - accuracy: 0.7335 - val_loss: 0.5478 - val_accuracy: 0.7612
Epoch 6/300
20/20 [==============================] - 0s 4ms/step - loss: 0.5496 - accuracy: 0.7512 - val_loss: 0.5275 - val_accuracy: 0.7687
Epoch 7/300
20/20 [==============================] - 0s 4ms/step - loss: 0.5325 - accuracy: 0.7544 - val_loss: 0.5119 - val_accuracy: 0.7649
Epoch 8/300


In [16]:
# 학습이 끝나서 모델이 만들어지면
# test.csv를 이용해서 결과 파일을 만들어야 함
# Kaggle에 제출하면 끝

test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICT Innovation Square/data/titanic_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICT Innovation Square/data/titanic_gender_submission.csv')

display(test.head())
display(submission.head())

# 테스트데이터 전처리
# 사용하지 않는 column 제거
test.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis=1, inplace=True)

# 성별 처리
gender_mapping = { 'male' : 0, 'female' : 1 }
test['Sex'] = test['Sex'].map(gender_mapping)

# 가족처리
test['Family'] = test['SibSp'] + test['Parch']
test.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# Embarked 결측치 처리
test['Embarked'] = test['Embarked'].fillna('Q')

# Embarked 문자를 숫자로 변환
embarked_mapping = { 'S' : 0, 'C' : 1, 'Q' : 2 }
test['Embarked'] = test['Embarked'].map(embarked_mapping)

# Age 결측치 처리
test['Age'] = test['Age'].fillna(test['Age'].mean())

# Age Binning 처리(Numerical Value -> Categorical Value)
test.loc[test['Age'] < 8, 'Age'] = 0
test.loc[(test['Age'] >= 8) & (test['Age'] < 20), 'Age'] = 1
test.loc[(test['Age'] >= 20) & (test['Age'] < 65), 'Age'] = 2
test.loc[test['Age'] >= 65, 'Age'] = 3

display(test)

predict = model.predict(test)

submission['Survived'] = predict
submission['Survived'] = submission['Survived'].astype('int')
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/ICT Innovation Square/data/sub.csv', index=False)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


,Pclass,Sex,Age,Embarked,Family
0,3,0,2.0,2,0
1,3,1,2.0,0,1
2,2,0,2.0,2,0
3,3,0,2.0,0,0
4,3,1,2.0,0,2
...,...,...,...,...,...
413,3,0,2.0,0,0
414,1,1,2.0,1,0
415,3,0,2.0,0,0
416,3,0,2.0,0,0


14/14 [==============================] - 0s 3ms/step
